<a href="https://colab.research.google.com/github/william-sharp/cs_216_project/blob/main/CS216_Colab_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

CS-216 Final Project

In [ ]:
import pandas as pd

url1 = "https://raw.githubusercontent.com/william-sharp/cs_216_project/refs/heads/main/fishing-industry-by-country-2024.json"
fish_industry_df = pd.read_json(url1)

url2 = "https://raw.githubusercontent.com/william-sharp/cs_216_project/refs/heads/main/fisheries_support_estimate.csv"
fish_support_estimate = pd.read_csv(url2)

url3 = "https://raw.githubusercontent.com/william-sharp/cs_216_project/refs/heads/main/travel_and_tourism_index_csv.csv"
tandt_index = pd.read_csv(url3, encoding = "ISO-8859-1", header = [0,1])
# Loop through the columns and join top and bottom column names if applicable
new_columns = []
for col in tandt_index.columns:
    top_level = col[0]
    bottom_level = col[1]
    new_columns.append(f"{top_level} - {bottom_level}" if "Unnamed" not in top_level else bottom_level)
tandt_index.columns = new_columns


url4 = "https://raw.githubusercontent.com/william-sharp/cs_216_project/refs/heads/main/ocean_health_index_scores.csv"
ocean_health_index = pd.read_csv(url4)

url5 = "https://raw.githubusercontent.com/william-sharp/cs_216_project/refs/heads/main/sustainable_fisheries_as_proportion_of_gdp.csv"
sust_fish_prop_gpd = pd.read_csv(url5)

url6 = "https://raw.githubusercontent.com/william-sharp/cs_216_project/refs/heads/main/employment_in_fisheries.csv"
employment_fish = pd.read_csv(url6)

In [ ]:
#Filter to get useful columns from fishing industry dataset
filtered_fish_industry_df = fish_industry_df[[column for column in fish_industry_df.columns if "2021" in column or column == "country"]]

#Filter to get useful columns from the estimated fishery support dataset
filtered_fish_support_estimate = fish_support_estimate[[column for column in fish_support_estimate.columns if column in [ "Country", "Variable", "Measure", "TIME_PERIOD", "OBS_VALUE" ]]]
filtered_fish_support_estimate = filtered_fish_support_estimate[filtered_fish_support_estimate["Measure"] == "US dollar"]
filtered_fish_support_estimate = filtered_fish_support_estimate.drop(columns=['Measure'])

#Filter to get useful columns from travel and tourism dataset
filtered_tandt_index = tandt_index[[column for column in tandt_index.columns if "2021" in column or column in ["Economy"]]]

#Filter to get useful columns and rows from ocean_health_index

ohi_index_mask = ocean_health_index["long_goal"].isin(["Mariculture (subgoal)", "Tourism & recreation", "Fisheries (subgoal)"])
ocean_health_index = ocean_health_index[ohi_index_mask]
ohi_index_mask_timeframe = ocean_health_index["dimension"].isin(["score"])
ocean_health_index = ocean_health_index[ohi_index_mask_timeframe]
filtered_ocean_health_index = ocean_health_index[[column for column in ocean_health_index.columns if column in ["scenario", "long_goal", "region_name", "value"]]]

#Filter to get useful columns from the sustainable fisheries as a proportion of GDP dataset
filtered_sust_fish_prop_gpd = sust_fish_prop_gpd[[column for column in sust_fish_prop_gpd.columns if column in ["Area", "TIME_PERIOD", "OBS_VALUE", ]]]

#Filter to get useful columns from fishery employment dataset
filtered_employment_fish = employment_fish[[column for column in employment_fish.columns if column in ["Reference area", "Working domain", "Sex", "Working status", "TIME_PERIOD", "OBS_VALUE"]]]


In [ ]:
filtered_fish_industry_df = filtered_fish_industry_df.rename(columns={"country": "Country", "FishingIndustry_TotalFisheriesProduction_tonnes_2021":"Total Fishery Production (tonnes)","FishingIndustry_CaptureFisheriesProduction_tonnes_2021":"Capture Fishery Production (tonnes)","FishingIndustry_AquacultureProduction_tonnes_2021":"Aquaculture Production (tonnes)"})
filtered_fish_support_estimate = filtered_fish_support_estimate.rename(columns={"TIME_PERIOD" : "Year", "OBS_VALUE": "Estimated Fishery Support (USD)", "Variable":"Support Type"})
filtered_tandt_index = filtered_tandt_index.rename(columns={"Economy": "Country"})
filtered_ocean_health_index = filtered_ocean_health_index.rename(columns={"region_name": "Country", "scenario":"Year", "value":"OHI Score", "long_goal":"OHI Goal"})
filtered_sust_fish_prop_gpd = filtered_sust_fish_prop_gpd.rename(columns={"Area": "Country", "TIME_PERIOD" : "Year", "OBS_VALUE": "Sustainable Fisheries as Proportion of GDP"})

filtered_employment_fish = filtered_employment_fish.rename(columns={"Reference area": "Country", "TIME_PERIOD" : "Year", "OBS_VALUE": "People Employed"})

In [ ]:
#filter all dataframes to use 2021 or 2020 data - most recent

filtered_fish_support_estimate = filtered_fish_support_estimate[filtered_fish_support_estimate["Year"] == 2020]
filtered_fish_support_estimate = filtered_fish_support_estimate[filtered_fish_support_estimate["Support Type"].isin(["I.A. Transfers based on input use"])]

ohi_index_mask_2 = filtered_ocean_health_index["Year"].isin([2021])
filtered_ocean_health_index = filtered_ocean_health_index[ohi_index_mask_2]

filtered_sust_fish_prop_gpd = filtered_sust_fish_prop_gpd[filtered_sust_fish_prop_gpd["Year"] == 2021]


filtered_employment_fish = filtered_employment_fish[filtered_employment_fish["Year"] == 2021]
filtered_employment_fish = filtered_employment_fish[filtered_employment_fish["Sex"] == "Total"]
filtered_employment_fish = filtered_employment_fish[filtered_employment_fish["Working status"] == "Total"]

filtered_employment_fish = filtered_employment_fish[filtered_employment_fish["Working domain"].isin(["Aquaculture","Fishing sector"])]



In [ ]:
#Delete year columns to not run into suffix errors
filtered_fish_support_estimate = filtered_fish_support_estimate.drop(columns=['Year'])
filtered_ocean_health_index = filtered_ocean_health_index.drop(columns=['Year'])
filtered_sust_fish_prop_gpd = filtered_sust_fish_prop_gpd.drop(columns=['Year'])
filtered_employment_fish = filtered_employment_fish.drop(columns=['Year', 'Sex','Working status'])

In [ ]:
# Perform inner joins step-by-step to ensure one row per Country-Year

merged_df = filtered_fish_industry_df.merge(filtered_fish_support_estimate, on="Country", how="outer")
merged_df = merged_df.merge(filtered_tandt_index, on="Country", how="outer")
merged_df = merged_df.merge(filtered_ocean_health_index, on="Country", how="right")
merged_df = merged_df.merge(filtered_sust_fish_prop_gpd, on="Country", how="left")
merged_df = merged_df.merge(filtered_employment_fish, on="Country", how="left")

pivot_df = merged_df.pivot_table(index='Country', columns='Working domain', values='People Employed', aggfunc='mean', fill_value=0)
pivot_df.reset_index(inplace=True)
merged_df = merged_df.merge(pivot_df, on="Country", how="outer")
merged_df = merged_df.drop(columns=['Working domain', 'People Employed'])
merged_df = merged_df.rename(columns = {"Aquaculture": "Persons Employed in Aquaculture", "Fishing sector":"Persons Employed in Fishing sector"})

merged_df



,Country,Total Fishery Production (tonnes),Capture Fishery Production (tonnes),Aquaculture Production (tonnes),Support Type,Estimated Fishery Support (USD),Travel & Tourism Development Index - 2021 Value,Travel & Tourism Development Index - 2021 Rank,Enabling Environment subindex - 2021 Value,Enabling Environment subindex - 2021 Rank,...,Environmental Sustainability pillar - 2021 Rank,Socioeconomic Resilience & Conditions pillar - 2021 Value,Socioeconomic Resilience & Conditions pillar - 2021 Rank,T&T Demand Pressure & Impact pillar - 2021 Value,T&T Demand Pressure & Impact pillar - 2021 Rank,OHI Goal,OHI Score,Sustainable Fisheries as Proportion of GDP,Persons Employed in Aquaculture,Persons Employed in Fishing sector
0,Albania,18283.0,9642.0,8641.0,NaN,NaN,3.8,72.0,4.7,60.0,...,27.0,4.7,39.0,3.4,111.0,Fisheries (subgoal),12.42,NaN,NaN,NaN
1,Albania,18283.0,9642.0,8641.0,NaN,NaN,3.8,72.0,4.7,60.0,...,27.0,4.7,39.0,3.4,111.0,Mariculture (subgoal),19.22,NaN,NaN,NaN
2,Albania,18283.0,9642.0,8641.0,NaN,NaN,3.8,72.0,4.7,60.0,...,27.0,4.7,39.0,3.4,111.0,Tourism & recreation,97.52,NaN,NaN,NaN
3,Algeria,83979.0,79200.0,4779.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Fisheries (subgoal),52.02,NaN,NaN,NaN
4,Algeria,83979.0,79200.0,4779.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Mariculture (subgoal),8.00,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
778,Western Sahara,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Mariculture (subgoal),0.00,NaN,NaN,NaN
779,Western Sahara,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Tourism & recreation,NaN,NaN,NaN,NaN
780,Yemen,131317.0,131307.0,10.0,NaN,NaN,2.6,116.0,2.8,116.0,...,117.0,2.6,116.0,3.7,84.0,Fisheries (subgoal),52.96,NaN,NaN,NaN
781,Yemen,131317.0,131307.0,10.0,NaN,NaN,2.6,116.0,2.8,116.0,...,117.0,2.6,116.0,3.7,84.0,Mariculture (subgoal),0.00,NaN,NaN,NaN


In [ ]:
#Andrws

sns.relplot(data=merged_df, x="Total Fishery Production (tonnes)", y="OHI Score")

In [ ]:
#Jerry

In [ ]:
#Oliver


In [ ]:
#will

hello this is will